In [ ]:
#| hide
from shyster.core import *

# shyster

> Add soft hyphens to HTML documents

As of now, I'm just playing around with nbdev.  But the problem I'm
trying to solve is that while I can set `hyphens: auto;` in CSS,
many browsers do a poor job of hyphenating Finnish.  Even if they
have Finnish hyphenation patterns, they often fail to recognise
compound words, which should be hyphenated at compound boundaries
(saippua-kauppias, not saip-pua-kaup-pias).  One solution is to set
`hyphens: manual;` and add soft hyphens at acceptable hyphenation
spots.


## Install

```sh
pip install https://github.com/jkseppan/shyster/
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2